In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
# geemap.update_package()

In [3]:
#list of plots in GA LTER dataset (not including broad creek)

ee.Initialize()

ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
]) ##Need to specify CRS? Not given, but assume NAD83 (crs ='EPSG:4269')

##These are average lat/longs of the site (see GIS file); as such, need zonal spectral data (not point data)
##Lat/longs calculated in GA Biomass dataset Jupyter script

##PC path:
ga_allplots = geemap.csv_to_pandas('C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots.csv') 
##plotcoords.csv is old

ga_allplots['Date'] = pd.to_datetime(ga_allplots[['Year', 'Month', 'Day']])
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots.csv' ##This will be what you load
ga_allplots.head()

##for GA, lats and longs may differ slightly between years (compare average lat/long to recorded lat/longs in each yr)


,Year,Day,Site,Zone,Plot,Latitude,Longitude,Month,Date
0,2000,10,4,1,1,31.451676,-81.365498,10,2000-10-10
1,2000,10,4,1,2,31.451548,-81.365560,10,2000-10-10
2,2000,10,4,1,3,31.451473,-81.365566,10,2000-10-10
3,2000,10,4,1,4,31.451373,-81.365579,10,2000-10-10
4,2000,10,4,1,5,31.451197,-81.366132,10,2000-10-10


In [4]:
##landsat 5 surface reflectance images - GA LTER

gaCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2012,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(gaCollectionLS5.size().getInfo())
gaimage0 = gaCollectionLS5.first()
props = geemap.image_props(gaimage0)
#props.getInfo()

100


In [5]:
##landsat 8 surface reflectance images - GA LTER
gaCollectionLS8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2012, 2016,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

l8_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B5', 'B4', 'B3'], 
             'gamma': 1.5} ##be very careful with analysis of l8 vs l5 data - different band numbers!

print(gaCollectionLS8.size().getInfo())

58


In [6]:
#landsat 7 surface reflectance images - GA LTER - might not be able to use due to striping
gaCollectionLS7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2012, 2012,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

In [7]:
##get list of image ids in filtered collection for L5, L7, and L8
gaCollectionLS5.aggregate_array('system:id').getInfo()
#gaCollectionLS8.aggregate_array('system:id').getInfo()
#gaCollectionLS7.aggregate_array('system:id').getInfo()

['LANDSAT/LT05/C01/T1_SR/LT05_016038_20021124',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20051015',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20061119',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20091127',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20101029',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20101114',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20111016',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20011027',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20031102',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20051123',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20061126',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20081030',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20101013',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20111101',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20080928',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20001102',
 'LANDSAT/LT05/C01/T1_SR/LT05_016038_20021007',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20060923',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20091001',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20101105',
 'LANDSAT/LT05/C01/T1_SR/LT05_017038_201

In [8]:
##Landsat Imagery
##Landsat times are in GMT
##Tide requirements: needs to be below 1.054 m at FORT PULASKI station (near Savannah) ID #8670870
##MHW = 2.173 m; MLW = 0.065 m
##In the future: Just use cloudless images and FLATS model to identify cloudless pictures

##landsat 5
ga00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20001024') ##TIDE = 0.2m ##some noise here; no spots near points though
##2000 sample dates: Oct 10-13,16-18

ga01 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20011027') ##TIDE = 0.5m
##2001 sample dates: Oct 5,8,9,15,16,17 

#ga02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20021007') ##TIDE = 2.1m XXXXXX, may be best?
ga02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20020921') ##TIDE = 1.6m XXXXXX, SEPTEMBER, clouds, still best choice
##2002 sample dates: Oct 14-20

#ga03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20030915') ##TIDE = 2.2m XXXXXX ##SEPTEMBER
ga03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20031102') ##TIDE = 0.9 m ##NOVEMBER
##2003 sample dates: NONE - no data for 2003

ga04 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20041104') ##TIDE = 1.77m XXXXXX ##NOVEMBER, still the best option
##2004 sample dates: Oct 22-25

ga05 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20051015') ##TIDE = 0.74m
##2005 sample dates: Oct 21-24,26,27

ga06 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20061002') ##TIDE = 0.74m
##2006 sample dates: Oct 14-17

ga07 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20070910') ##TIDE = 0.85m ##SEPTEMBER
##2007 sample dates: Oct 20-24

#ga08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20081030') ##TIDE = 2.1m XXXXXX
ga08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20080928') ##TIDE = 1.2m XXX, SEPTEMBER, still best option
##2008 sample dates: Oct 25-28

ga09 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20091001') ##TIDE = 0.58m
##2009 sample dates: Oct 14-17

#ga10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20101029') ##TIDE = 1.93m XXXXXX
ga10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20101105') ##TIDE = 1.02m ##NOVEMBER
##2010 sample dates: Oct 18-21

#ga11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20111016') ##TIDE = 2.26m XXXXXX
ga11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20111023') ##TIDE = 0.1m
##2011 sample dates: Oct 24-27

##landsat 7
ga12 = ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_016038_20120924') ##TIDE = 0.74m, L7 XXXXXX SEPTEMBER
##2012 sample dates: Oct 12-15

##landsat 8
#ga13 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20131012') ##TIDE = 1.7m XXXXXX
ga13 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20131113') ##TIDE = 0.56m ##NOVEMBER
##2013 sample dates: Oct 13-16,18

#ga14 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20141008') ##TIDE = 1.25m XXXXXX
ga14 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20140922') ##TIDE = 0.79m
##2014 sample dates: Oct 15-18,20,21

#ga15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20151018') ##TIDE = 2.4m XXXXXX
ga15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20150909') ##TIDE = 0.11 ##CLOUDS
##2015 sample dates: Oct 15,20-25

#ga16 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20161029') ##TIDE = 1.09m XXXXXX; May be ok? Check flooded pixels
ga16 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20160911') ##TIDE = 0.72 ##SEPTEMBER, check sample date
##2016 sample dates: Oct 23-27

ga17 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20171016') ##TIDE = 0.28m
##2017 sample dates: Oct 14,16,18-20,23

ga18 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20181003') ##TIDE = 1.15m XXXXXX; May be ok? Check flooded pixels
##2018 sample dates: Oct 12-17,19
#ga18 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20180908') ##TIDE = 0.43m ##so many clouds, however; might be \
                                                            ## to use higher tide image? And calculate flooded pixels

ga19 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20190927') ##TIDE = 0.69m ##SEPTEMBER
##2019 sample dates: Oct 4,5,7-9,11

##determining cloud cover
cloudiness = ga00.get('CLOUD_COVER')
#print('CLOUD_COVER: ', cloudiness.getInfo())

#Map.addLayer(ga15, l8_param, "l82015")
#Map.addLayer(ga12, vis_param, "l72012")

props = geemap.image_props(ga00)
props.getInfo()

##Need flooded pixel analysis (either TMII or FLATS for Landsat)

{'CLOUD_COVER': 5,
 'CLOUD_COVER_LAND': 5,
 'EARTH_SUN_DISTANCE': 0.994431,
 'ESPA_VERSION': '2_19_0c',
 'GEOMETRIC_RMSE_MODEL': 3.781,
 'GEOMETRIC_RMSE_MODEL_X': 2.359,
 'GEOMETRIC_RMSE_MODEL_Y': 2.955,
 'IMAGE_DATE': '2000-10-24',
 'IMAGE_QUALITY': 9,
 'LANDSAT_ID': 'LT05_L1TP_017038_20001024_20160918_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1474200080000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.5.0',
 'SATELLITE': 'LANDSAT_5',
 'SENSING_TIME': '2000-10-24T15:39:31.2120380Z',
 'SOLAR_AZIMUTH_ANGLE': 149.348083,
 'SOLAR_ZENITH_ANGLE': 49.074329,
 'SR_APP_VERSION': 'LEDAPS_3.2.0',
 'WRS_PATH': 17,
 'WRS_ROW': 38,
 'system:asset_size': '342.521984 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'sr_atmos_opacity',
  'sr_cloud_qa',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20001024',
 'system:index': 'LT05_017038_20001024',
 'system:time_end': '2000-10-24 15:39:31',
 'system:time_start': '2000-10-

In [9]:
##Mapping
##Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(ga01, vis_param, "Landsat5SR")
#Map.addLayer(ga13, l8_param, "Landsat8SR")

Map.centerObject(ga_pts, 10)
#Map.set_plot_options(add_marker_cluster = True)

##add feature collection points for each plot
Map.addLayer(ga_pts, {}, "avg_plots")

Map
##what is with all the spots on ga00? Something wrong with image, salt & pepper noise?

Map(center=[31.434120927658846, -81.34324597505625], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
##Adding every plot coordinate, not just averages
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots.csv'

##^^Might still be useful, use 'fc = ' function below
##Map.add_xy_data(allplots_fc, x="Longitude", y="Latitude", layer_name="csv to ee", to_ee=True)

##Adding average coordinates - change from above (allplots_fc), which is every single plot
allplots_av = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/allplots_av.csv'

fc = geemap.csv_to_ee(allplots_av, latitude = "Latitude", longitude = "Longitude")

fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc, {}, "fc") ##with average plots, some variability in location. With all plots, many different (overlapping) locations

Map.addLayer(fc_all, {}, "fc_all")

# in_csv = 'C:/Users/arj26323/Documents/Data/Biomass datasets/plotcoord.csv'
# df_fc = geemap.csv_to_pandas(in_csv)

In [11]:
##Water masking

##Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')

##Select the land/water mask.
datamask = hansenImage.select('datamask')

##Create a binary mask.
watermask = datamask.eq(1)

#watermasked00 = ga00.updateMask(watermask)
#Map.addLayer(watermasked00, vis_param, 'masked')

##Works! Although it is not perfect. Make sure to turn off background layers to see.
##See if there are any other water masks that perform better. Small creeks not masked at all

In [12]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
##Seems to change reflectance values slightly (besides the dividing by 10000 part)
def maskL5sr(image):
  ## Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

##Map the function over image collection
collection = gaCollectionLS5 \
    .map(maskL5sr)

composite = collection.median() ##what exactly is this doing? median pixel value?

In [13]:
##water-, cloud-, and cloud shadow-masked images - GA LTER :D
gamasked00 = ee.Image(maskL5sr(ga00)).updateMask(watermask)
gamasked01 = ee.Image(maskL5sr(ga01)).updateMask(watermask)
gamasked02 = ee.Image(maskL5sr(ga02)).updateMask(watermask)
gamasked03 = ee.Image(maskL5sr(ga03)).updateMask(watermask) ##no data collected in Pennings dataset this year
gamasked04 = ee.Image(maskL5sr(ga04)).updateMask(watermask)
gamasked05 = ee.Image(maskL5sr(ga05)).updateMask(watermask)
gamasked06 = ee.Image(maskL5sr(ga06)).updateMask(watermask)
gamasked07 = ee.Image(maskL5sr(ga07)).updateMask(watermask)
gamasked08 = ee.Image(maskL5sr(ga08)).updateMask(watermask)
gamasked09 = ee.Image(maskL5sr(ga09)).updateMask(watermask)
gamasked10 = ee.Image(maskL5sr(ga10)).updateMask(watermask)
gamasked11 = ee.Image(maskL5sr(ga11)).updateMask(watermask)
gamasked12 = ee.Image(maskL5sr(ga12)).updateMask(watermask)
gamasked13 = ee.Image(maskL5sr(ga13)).updateMask(watermask)
gamasked14 = ee.Image(maskL5sr(ga14)).updateMask(watermask)
gamasked15 = ee.Image(maskL5sr(ga15)).updateMask(watermask)
gamasked16 = ee.Image(maskL5sr(ga16)).updateMask(watermask)
gamasked17 = ee.Image(maskL5sr(ga17)).updateMask(watermask)
gamasked18 = ee.Image(maskL5sr(ga18)).updateMask(watermask)
gamasked19 = ee.Image(maskL5sr(ga19)).updateMask(watermask)

##Check masked images:
Map.addLayer(gamasked01, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "masked")

##Creating an image collection:
ga_finalcollection = ee.ImageCollection.fromImages([gamasked00, gamasked01, gamasked02, gamasked03, gamasked04, \
                                                    gamasked05, gamasked06, gamasked07, gamasked08, gamasked09, \
                                                    gamasked10, gamasked11, gamasked12, gamasked13, gamasked14, \
                                                    gamasked15, gamasked16, gamasked17, gamasked18, gamasked19])

Map.addLayer(gamasked00, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "masked00")
Map.addLayer(ga_pts)

# ga_finalcollection = ee.ImageCollection.fromImages([gamasked00, gamasked01, gamasked02, gamasked04, \
#                                                     gamasked05, gamasked06, gamasked07, gamasked08, gamasked09, \
#                                                     gamasked10, gamasked11, gamasked12, gamasked13, gamasked14, \
#                                                     gamasked15, gamasked16, gamasked17, gamasked18, gamasked19])

##second one is without 2003, when there were no biomass data collected by Pennings

##Unmasked collection, just to check masked/unmasked pixel values at 30m scale - they match (just /10000)
ga_unmaskedcollection = ee.ImageCollection.fromImages([ga00, ga01, ga02, ga03, ga04, \
                                                    ga05, ga06, ga07, ga08, ga09, \
                                                    ga10, ga11, ga12, ga13, ga14, \
                                                    ga15, ga16, ga17, ga18, ga19])

In [17]:
##Adding DEM, extracting DEM raster values
import localtileserver
dem = 'E:/Elevation/USGS Sapelo/USGS_1_n32w082.tif'
dem2 = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m. Also I don't know what the data looks like
dem3 = ee.Image('USGS/SRTMGL1_003') ##1 arc second, or 30 m, and large coverage. Elevation is in meters (from GDEM datum?)

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.add_raster(dem, colormap = "terrain", layer_name='DEM')
Map.addLayer(dem3, dem_params, 'SRTM DEM', True, 1)
Map.addLayer(dem2, dem_params, '10m DEM', True, 1)

##Exporting pixel values for DEM, all plots!
in_fc = fc_all 
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'DEM_vals_10m.csv')
# geemap.extract_values_to_points(in_fc, dem2, out_csv)

In [15]:
##Exporting pixel values from points
coords_year = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/Coords_year/NoFlower/coords_2019.csv'
##NoFlower folder is identical to Coords_year folder, with flowering status removed (confuses csv_to_ee function)
fc_coords = geemap.csv_to_ee(coords_year, latitude = "Latitude", longitude = "Longitude")
# in_fc = fc_coords ##changed from ga_points - getting pixel values for individual plots

Map.addLayer(fc_coords, {}, "fc_coords")

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'test200.csv')
#out_shp = os.path.join(out_dir, 'points.shp') ##shapefile exporting

# geemap.extract_values_to_points(fc_coords, gamasked19, out_csv) ##only works for single images - maybe for loop?

## From youtube comments: I would like to use this function for large time series data sets; Is it possible \
##to make a loop over a large ImageCollection and storing all results in one file? 

##Response: You just need to convert an ImageCollection to Image  (e.g., image=collection.toBands()) \
## and then follow the same procedure in the video to extract time-series pixel values.

# xyx = ga_finalcollection.toBands()##Does this combine all bands within image collection?
# geemap.extract_values_to_points(fc_all, gamasked19, out_csv) ##compare to 'df' in GA-yearly file

##With 'all points', all points for each year (overlapping) are included

In [ ]:
##Exporting pixel values - zonal statistics: Average 3x3 grid for Landsat \
##(radius = 30 or 90 m - is the 'scale' option in zonal_statistics radius??? ALSO: Be careful with upland veg and water)
##For now, stick with scale = 30m - gets single-pixel information. Will have to think about this for VA though

out_dir = os.path.expanduser('~/Downloads')
out_ga_stats = os.path.join(out_dir, 'ga_avtest.csv')  

# geemap.zonal_statistics(in_value_raster = ga_finalcollection, in_zone_vector = fc, \
#                 out_file_path = out_ga_stats, statistics_type='MEAN', \
#                 scale = 30)

In [ ]:
##Export CSV file to then load as feature class
in_fc = ga_allplots
out_dir = os.path.expanduser('~/Documents/Data/Biomass datasets/Sapelo')
out_csv = os.path.join(out_dir, 'plotcoord_fc.csv')
#ga_allplots.to_csv('~/Documents/Data/Biomass datasets/BMA_fc.csv', index = False)
#print(df)